In [ ]:
# SSHClient类示例

# 通过账号密码登录
import paramiko

hostname = "172.16.100.8"
# hostname = "127.0.0.1"
username = "root"
password = "lh.com"

paramiko.util.log_to_file('syslogin.log') 
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
# ssh.load_system_host_keys()

ssh.connect(hostname=hostname, username=username, password=password)

stdin, stdout, stderr = ssh.exec_command("free -m")
print stdout.read()
print stderr.read()

ssh.close()


In [ ]:
# 通过私钥登录
import paramiko

hostname = "172.16.100.8"
# hostname = "127.0.0.1"
username = "root"

paramiko.util.log_to_file('syslogin.log') 
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)

pkey = "/root/.ssh/id_rsa"
key = paramiko.RSAKey.from_private_key_file(pkey, password="lh.com")  # 有密码时解密私钥
ssh.connect(hostname=hostname, username=username, pkey=key)

stdin, stdout, stderr = ssh.exec_command("free -m")
print stdout.read()
print stderr.read()

ssh.close()


In [ ]:
# SFTPClient 类示例
import paramiko

hostname = "172.16.100.8"
# hostname = "127.0.0.1"
username = "root"
password = "lh.com"
port = 22

t = paramiko.Transport((hostname, port))
t.connect(username=username, password=password)
sftp = paramiko.SFTPClient.from_transport(t)

sftp.put("/root/upload.txt", "/tmp/upload.txt") # 上传文件
sftp.get("/tmp/download.txt", "/root/download.txt") # 下载文件
sftp.mkdir("/tmp/userdir", 0755)  # 创建目录
sftp.rmdir("/tmp/userdirold")  # 删除目录
print sftp.stat("/tmp/upload.txt") # 打印文件信息
print sftp.listdir("/tmp") # 打印目录列表

t.close()

In [ ]:
# paramiko 应用示例

# 1. 实现密钥方式登录远程主机

import paramiko

hostname = "172.16.100.8"
username = "root"

paramiko.util.log_to_file('syslogin.log') 
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)

pkey = "/root/.ssh/id_rsa"
key = paramiko.RSAKey.from_private_key_file(pkey, password="lh.com")  # 有密码时解密私钥
ssh.connect(hostname=hostname, username=username, pkey=key)

stdin, stdout, stderr = ssh.exec_command("free -m")
print stdout.read()
print stderr.read()

ssh.close()


In [76]:
# 2. 堡垒机模式下，远程命令执行

import paramiko
import sys

blip = "172.16.100.8"   # 定义堡垒机的信息
bluser = "root"
blpasswd = "lh.com"

hostname = "172.16.100.7"  # 定义业务服务器信息
username = "root"
password = "lh.com"

ssh = paramiko.SSHClient()  # ssh 登录堡垒机
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
ssh.connect(hostname=blip, username=bluser, password=blpasswd)

channel = ssh.invoke_shell()  # 创建回话，开启命令调用
channel.settimeout(10)        # 会话命令执行超时时间，单位为秒

buff = ""
resp = ""
passinfo = "\'s password: "   # 输入服务器密码的前标志串
paramiko.util.log_to_file('syslogin.log')

channel.send("ssh " + username + "@" + hostname + "\n")  # 执行ssh 登录业务主机
resp = channel.recv(9999)

while not buff.endswith(passinfo):  # ssh登录的提示信息判断，输出串尾含有“\'s password: ”时
    # 退出while 循环
    try: 
        resp = channel.recv(9999)
    except Exception, e:
        print "Error info: %s connnection time." % (str(e))
        channel.close()
        ssh.close()
        sys.exit()
    buff += resp
    if not buff.find("yes/no") == -1:  # 输出串尾含有“yes/no”时，发送“yes”并回车
        channel.send("yes\n")
        buff = ""

channel.send(password + "\n")   # 发送业务主机密码

buff = ""
while not buff.endswith("# "): # 输出串尾为“# ”时说明校验通过并退出while循环
    resp = channel.recv(9999)
    if not resp.find(passinfo) == -1:  # 输出串尾含有“\'s password: ”时说明密码不正确，
                                        # 要求重新输入
        print "Error info: Authenticatin failed."
        channel.close()  # 关闭连接对象后退出
        ssh.close()
        sys.exit()
    buff += resp

channel.send("ifconfig\n")  # 认证通过后发送ifconfig命令来查看结果

# 查看结果
buff = ""
try:
    while buff.find("# ") == -1:
        resp = channel.recv(9999)
        buff += resp
except Exception, e:
    print "error info:" + str(e)

print buff  # 打印输出串

channel.close()
ssh.close()

ifconfig
eth0      Link encap:Ethernet  HWaddr 00:0C:29:A6:57:18  
          inet addr:172.16.100.7  Bcast:172.16.255.255  Mask:255.255.0.0
          inet6 addr: fe80::20c:29ff:fea6:5718/64 Scope:Link
          UP BROADCAST RUNNING MULTICAST  MTU:1500  Metric:1
          RX packets:1280 errors:0 dropped:0 overruns:0 frame:0
          TX packets:1034 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1000 
          RX bytes:185322 (180.9 KiB)  TX bytes:211000 (206.0 KiB)

eth1      Link encap:Ethernet  HWaddr 00:0C:29:A6:57:22  
          inet addr:192.168.243.128  Bcast:192.168.243.255  Mask:255.255.255.0
          inet6 addr: fe80::20c:29ff:fea6:5722/64 Scope:Link
          UP BROADCAST RUNNING MULTICAST  MTU:1500  Metric:1
          RX packets:1620 errors:0 dropped:0 overruns:0 frame:0
          TX packets:1872 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1000 
          RX bytes:149009 (145.5 KiB)  TX bytes:159872 (156.1 KiB)
    

In [94]:
# 3. 堡垒机模式下，远程文件上传
import paramiko
import sys

blip = "172.16.100.8"   # 定义堡垒机的信息
bluser = "root"
blpasswd = "lh.com"

hostname = "172.16.100.7"  # 定义业务服务器信息
username = "root"
password = "lh.com"

tmpdir = "/tmp"
remotedir = "/tmp"
localpath = "/root/testUpload.txt"  # 本地源文件路径
tmppath = tmpdir + "/testUpload.txt"  # 堡垒机临时路径
remoterpath = remotedir + "/testUpload.txt" # 业务主机目标路径

port = 22
passinfo = "\'s password: "
paramiko.util.log_to_file("syslogin.log")

t = paramiko.Transport((blip, port))
t.connect(username=bluser, password=blpasswd)
sftp = paramiko.SFTPClient.from_transport(t)

sftp.put(localpath, tmppath)  # 上传本地源文件到堡垒机临时路径
sftp.close()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
ssh.connect(hostname=blip, username=bluser, password=blpasswd)

channel = ssh.invoke_shell()
channel.settimeout(10)

buff = ""
resp = ""

# scp 中转目录文件到目标主机
# scp tmppath root@172.16.100.7:remotepath

channel.send("scp " + tmppath + " " + username + "@" + hostname + ":" + remoterpath + "\n")

while not buff.endswith(passinfo):
    try:
        resp = channel.recv(9999)
    except Exception, e:
        print "Error info: %s connection time." % (str(e))
        channel.close()
        ssh.close()
        sys.exit()
    buff += resp
    if not buff.find("yes/no") == -1:
        channel.send("yes\n")
        buff = ""
channel.send(password + "\n")

buff = ""
while not buff.endswith("# "):
    resp = channel.recv(9999)
    if not resp.find(passinfo)==-1:
        print "Erro info: Authentication failed."
        channel.close()
        ssh.close()
        sys.exit()
        
    buff += resp
    
print buff
channel.close()
ssh.close()


testUpload.txt                                100%   11     0.0KB/s   00:00    
[root@linux-master01 ~]# 
